In [3]:
!pip install fitz PyMuPDF

     |████████████████████████████████| 6.2 MB 10.5 MB/s eta 0:00:01


In [70]:
import os
import fitz
import re
from glob import glob
import pandas as pd

In [62]:
### From parse_in_rolls

COLUMNS = ["number","id", "elector_name", "father_or_husband_name", "relationship", "house_no", "age", "sex", "ac_name", "parl_constituency", "part_no", "year", "state", "filename", "main_town", "police_station", "mandal", "revenue_division", "district", "pin_code", "polling_station_name", "polling_station_address", "net_electors_male", "net_electors_female", "net_electors_third_gender", "net_electors_total","original_or_amendment"]

In [122]:
rows = []
for fn in glob('../../parse_in_rolls/data/haryana/*.PDF'):
    doc = fitz.open(fn)
    for p in doc:
        text = p.getText()
        for m in re.finditer(r'(?<=\n)(.*?\n\d+\n.*?\nनाम :(.*?\nहै।))', '\n' + text, flags=re.DOTALL):
            box = m.group(1)
            elector_name = re.findall(r'(.*?)\d+', box, flags=re.S)[0].strip().replace('\n', ' ')
            number = re.findall(r'.*?(\d+)', box, flags=re.S)[0].strip()
            _id = re.findall(r'.*?\d+\n(.*?)\n', box, flags=re.S)[0].strip()
            a = re.findall(r'नाम :\n(.*?)\nमकान  नं. :', box, flags=re.S)[0].strip().split(':')
            relationship = a[0].strip()
            father_or_husband_name = a[1].strip()
            house_no = re.findall(r'मकान  नं. :(.*?)आयु', box, flags=re.S)[0].strip()
            age = re.findall(r'आयु :(.*?)िलग', box, flags=re.S)[0].strip()
            sex = re.findall(r'लग : (.*?)फोटो', box, flags=re.S)[0].strip()
            data = {'filename': os.path.basename(fn),
                    'number': number,
                    'id': _id,
                    'elector_name': elector_name,
                    'father_or_husband_name': father_or_husband_name,
                    'relationship': relationship,
                    'house_no': house_no,
                    'age': age,
                    'sex': sex,
                   }
            rows.append(data)

In [123]:
df = pd.DataFrame(rows)
df

,filename,number,id,elector_name,father_or_husband_name,relationship,house_no,age,sex
0,CMB0880189.PDF,1,BFS2536829,सुरेश चंद,रतन िसह,िपता,.,52,पुरुष
1,CMB0880189.PDF,2,BFS2536803,मोनू,रतन िसह,िपता,.,34,पुरुष
2,CMB0880189.PDF,3,BFS2536795,शिश,रतन िसह,िपता,.,31,मिहला
3,CMB0880189.PDF,4,BFS2536811,अिनल,रतन िसह,िपता,.,31,पुरुष
4,CMB0880189.PDF,5,NAH1151406,रेनू भार�वाज,दीपक भार�ाज,पित,0,32,मिहला
...,...,...,...,...,...,...,...,...,...
26990,CMB0690082.PDF,888,BTX1467190,मदन लाल,लालमन,िपता,225ख,68,पुरुष
26991,CMB0690082.PDF,889,BTX1661560,शिमर्ला देवी,राजेश कुमार,पित,225ख,38,मिहला
26992,CMB0690082.PDF,890,SHQ0875336,मनीष,बीर�दर् िसह,िपता,236,20,पुरुष
26993,CMB0690082.PDF,891,SHQ0280586,सुरे�द्र,अजमेर िसह,िपता,255,28,पुरुष


In [124]:
df.relationship.unique()

array(['िपता', 'पित', 'माता', '......'], dtype=object)

In [125]:
df.sex.unique()

array(['पुरुष', 'मिहला'], dtype=object)

In [126]:
df.age.unique()

array(['52', '34', '31', '32', '27', '26', '25', '24', '23', '28', '57',
       '35', '47', '44', '51', '40', '60', '58', '64', '62', '54', '38',
       '45', '41', '59', '37', '56', '50', '36', '74', '68', '43', '76',
       '72', '66', '46', '61', '48', '42', '71', '39', '55', '75', '67',
       '49', '30', '53', '33', '29', '63', '69', '65', '77', '20', '81',
       '80', '70', '22', '89', '84', '73', '85', '21', '79', '83', '87',
       '19', '82', '78', '99', '95', '90', '92', '91', '94', '93', '96',
       '88', '86', '97', '100', '102', '108', '112', '98', '101', '103',
       '111', '113'], dtype=object)

In [128]:
df.to_csv('haryana.csv', index_label='index')

In [129]:
text

'नामावली पर्रुप\nनामावली पहचान\nिनवार्चक� की संख्या\nपुरुष\nमिहला\nअन्य\nकुल\nमूल पर्ित \nमूल नामावली\nनये पिरसीिमत िनवार्चन क्षेतर्� के िवस्तारानुसार \nवषर्  2017 तक के सभी अनुपूरक� सिहत एकीकृत \nवषर् 2018 की फोटो युक्त िनवार्चक नामावली\n455\n437\n0\n892\n455\n437\n0\n892\nिनवार्चक पंजीयन अिधकारी �ारा जारी \nकुल  33  पृ�� का पृ�  33\n'